In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news-data.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)
y=df['label']

In [6]:
X.shape

(18285, 4)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [10]:
voc_size=5000

In [11]:
messages=X.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tusha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3443, 511, 4336, 440, 4803, 756, 744, 3204, 2941, 4814],
 [4723, 2468, 2727, 2833, 3274, 1087, 664],
 [4961, 3580, 2639, 571],
 [3512, 58, 4388, 374, 930, 3965],
 [946, 3274, 3667, 1137, 2192, 602, 3274, 2146, 2862, 1810],
 [2298,
  59,
  2931,
  4656,
  4946,
  844,
  453,
  330,
  473,
  4568,
  2585,
  3689,
  2517,
  239,
  664],
 [4561, 2276, 3313, 1450, 380, 3701, 1623, 335, 1145, 2640, 158],
 [197, 3085, 619, 527, 3965, 733, 844, 1575, 1145, 2640, 158],
 [3907, 357, 4391, 4690, 2173, 3256, 1464, 4395, 844, 4247],
 [2295, 1794, 2160, 1457, 35, 1836, 2076, 929],
 [1930, 923, 4333, 157, 949, 4086, 445, 2119, 285, 2646, 2784],
 [374, 1902, 4803, 3256, 844, 3965],
 [1333, 2149, 3728, 432, 2962, 1273, 3235, 4284, 3911],
 [2902, 238, 2891, 3163, 210, 508, 4337, 1145, 2640, 158],
 [3362, 2798, 1140, 4970, 61, 1145, 2640, 158],
 [1216, 699, 1715, 914, 52, 1901, 1182, 3437, 2870, 3907],
 [2479, 3286, 2468],
 [3678, 3886, 4616, 2710, 844, 52, 4858, 664],
 [2728, 4258, 2727, 2631, 1496, 4

In [23]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3204 2941 4814]
 [   0    0    0 ... 3274 1087  664]
 [   0    0    0 ... 3580 2639  571]
 ...
 [   0    0    0 ... 1145 2640  158]
 [   0    0    0 ...  827 2076 3601]
 [   0    0    0 ... 1473 2181 3055]]


In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final,test_size=0.33,random_state=42)

In [28]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 19ms/step - loss: 0.3410 - accuracy: 0.8346 - val_loss: 0.2019 - val_accuracy: 0.9138
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1415 - accuracy: 0.9429 - val_loss: 0.2028 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1019 - accuracy: 0.9624 - val_loss: 0.2297 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0713 - accuracy: 0.9754 - val_loss: 0.2585 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0506 - accuracy: 0.9848 - val_loss: 0.2935 - val_accuracy: 0.9069
Epoch 6/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0362 - accuracy: 0.9891 - val_loss: 0.3337 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0248 - accuracy: 0.9927 - val_loss: 0.4331 - val_accuracy: 0.9052

In [29]:
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 20ms/step - loss: 0.3476 - accuracy: 0.8384 - val_loss: 0.2025 - val_accuracy: 0.9140
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1528 - accuracy: 0.9398 - val_loss: 0.2003 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1048 - accuracy: 0.9609 - val_loss: 0.2060 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0740 - accuracy: 0.9746 - val_loss: 0.2644 - val_accuracy: 0.9089
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0501 - accuracy: 0.9842 - val_loss: 0.2893 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0374 - accuracy: 0.9880 - val_loss: 0.3149 - val_accuracy: 0.9120
Epoch 7/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0282 - accuracy: 0.9915 - val_loss: 0.3132 - val_accuracy: 0.9117

In [38]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [42]:
threshold = 0.5
y_pred = np.where(y_pred >= threshold, 1, 0)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(y_test,y_pred)

0.903728251864126

In [45]:
confusion_matrix(y_test,y_pred)

array([[3155,  264],
       [ 317, 2299]], dtype=int64)